In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sys import platform
import io
import datetime
import requests
import sys
import ephem # Might need to run pip3 install ephem

In [2]:
# Data location
data_string = "/Users/erikamiller/Desktop/Pied Piper/Data/Skagit/"
output_string = "/Users/erikamiller/Desktop/Pied Piper/Output/Skagit/"

---------------------------------------------------------------------------------------------------------------------
                                                    For scoop file!
---------------------------------------------------------------------------------------------------------------------

In [3]:
# Read the dataset

year = 2015
csv_name = data_string + str(year) + "_skagit_mainstem_scoop.csv"
df_2015_scoop = pd.read_csv(csv_name, header = 0)

df_2015_scoop.head()

,Unnamed: 0,StartDate,EndDate,StartTime,EndTime,chinook0_wild_num,chinook0_hatchery_num,chinook1_wild_num,chinook1_hatchery_num,chum0_wild_num,...,coho1_wild_num,coho1_hatchery_num,steelheadsmolt_wild_num,sockeye0_wild_num,sockeye1_wild_num,pink0_wild_num,start_datetime,end_datetime,In,midpoint
0,1,2015-01-29,2015-01-30,1899-12-30 17:30:00,1899-12-30 07:00:00,101,0,1,0,0,...,0,0,0,0,0,0,2015-01-29 17:30:00,2015-01-30 07:00:00,13.5,2015-01-30 00:15:00
1,2,2015-01-30,2015-01-31,1899-12-30 17:30:00,1899-12-30 07:30:00,112,0,0,0,0,...,0,0,0,0,0,0,2015-01-30 17:30:00,2015-01-31 07:30:00,14.0,2015-01-31 00:30:00
2,3,2015-01-31,2015-02-01,1899-12-30 17:30:00,1899-12-30 08:00:00,51,0,0,0,0,...,0,0,0,0,0,0,2015-01-31 17:30:00,2015-02-01 08:00:00,14.5,2015-02-01 00:45:00
3,4,2015-02-01,2015-02-02,1899-12-30 20:30:00,1899-12-30 08:00:00,40,0,0,0,0,...,0,0,0,0,0,0,2015-02-01 20:30:00,2015-02-02 08:00:00,11.5,2015-02-02 02:15:00
4,5,2015-02-02,2015-02-02,1899-12-30 08:00:00,1899-12-30 17:30:00,3,0,0,0,0,...,0,0,0,0,0,0,2015-02-02 08:00:00,2015-02-02 17:30:00,9.5,2015-02-02 12:45:00


In [4]:
# Rename columns
df_year = df_2015_scoop.rename(columns={'EndDate': "Up_date",
                                        'StartTime': "Down_time",
                                        'EndTime': "Up_time",
                                        'start_datetime': "Down",
                                        'end_datetime': "Up"})

df_year.columns   # List the columns

Index(['Unnamed: 0', 'StartDate', 'Up_date', 'Down_time', 'Up_time',
       'chinook0_wild_num', 'chinook0_hatchery_num', 'chinook1_wild_num',
       'chinook1_hatchery_num', 'chum0_wild_num', 'coho0_wild_num',
       'coho1_wild_num', 'coho1_hatchery_num', 'steelheadsmolt_wild_num',
       'sockeye0_wild_num', 'sockeye1_wild_num', 'pink0_wild_num', 'Down',
       'Up', 'In', 'midpoint'],
      dtype='object')

In [5]:
# Subset the data
df_year_subset = df_year[['Down_time', 'Up_date', 'Up_time', 'midpoint', 'In', 'Up', 'Down',
                          'chinook0_wild_num', 'chinook0_hatchery_num',
                          'chinook1_wild_num', 'chinook1_hatchery_num',
                          'chum0_wild_num',
                          'coho0_wild_num',
                          'coho1_wild_num', 'coho1_hatchery_num',
                          'pink0_wild_num',
                          'sockeye1_wild_num', 'sockeye0_wild_num',
                          'steelheadsmolt_wild_num']]

In [6]:
# Format 'midpoint' to a date-time stamp
df_year_subset['midpoint'] = pd.to_datetime(df_year_subset['midpoint'])

# Checking whether the midpoint of the time when trap was in during day or night
def categorize_daytime(dt):
    if dt.hour >= 6 and dt.hour < 18:
        return 'Day'
    else:
        return 'Night'
df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)

/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/385244392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year_subset['midpoint'] = pd.to_datetime(df_year_subset['midpoint'])
/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/385244392.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)


In [7]:
df_day = df_year_subset[df_year_subset['daytime_category']=="Day"]
df_night = df_year_subset[df_year_subset['daytime_category']=="Night"]

# Need to have one row for every date
df_dates_day = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2015-01-28 12:00:00', 
                                       end='2015-07-14 12:00:00')})
df_dates_night = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2015-01-28 00:00:00', 
                                       end='2015-07-14 00:00:00')})

df_merged_day = pd.merge_asof(df_dates_day, 
              df_day, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

df_merged_night = pd.merge_asof(df_dates_night, 
              df_night, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

In [8]:
# New columns
df_merged_day['midpoint_doy'] = df_merged_day['midpoint'].dt.dayofyear
df_merged_day['doy'] = df_merged_day['date_range'].dt.dayofyear
df_merged_day['year'] = df_merged_day['date_range'].dt.year
df_merged_day['daytime_category'] = 'day'
df_merged_day['Date'] = pd.to_datetime(df_merged_day['date_range'].dt.date)

df_merged_night['midpoint_doy'] = df_merged_night['midpoint'].dt.dayofyear
df_merged_night['doy'] = df_merged_night['date_range'].dt.dayofyear
df_merged_night['year'] = df_merged_night['date_range'].dt.year
df_merged_night['daytime_category'] = 'night'
df_merged_night['Date'] = pd.to_datetime(df_merged_night['date_range'].dt.date)

In [9]:
#------------------------------------------------------------------------------------------------------------------
# Testing: checking where in the text that needs to be deleted
#------------------------------------------------------------------------------------------------------------------

# Getting temp and flow data from USGS
year = '2015'
url = 'https://waterdata.usgs.gov/nwis/dv?cb_00010=on&cb_00060=on&cb_00065=on&format=rdb&site_no=12178000&legacy=&referred_module=sw&period=&begin_date=2015-01-01&end_date=2015-12-16'

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# Keep changing numbers till the last character becomes "\n"
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(text[0:1951])

# # Testing so that the column names are lined up accordingly
# text_data = text[1951:] # Keep anything after the text in position 1951 (where data starts)
# test = pd.read_csv(io.StringIO(text_data), sep = '\t', header = None)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(test)

"# ---------------------------------- WARNING ----------------------------------------\n# Some of the data that you have obtained from this U.S. Geological Survey database\n# may not have received Director's approval. Any such data values are qualified\n# as provisional and are subject to revision. Provisional data are released on the\n# condition that neither the USGS nor the United States Government may be held liable\n# for any damages resulting from its use.\n#\n# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement\n#\n# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output\n# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals\n#\n# Contact:   gs-w_waterdata_support@usgs.gov\n# retrieved: 2023-11-16 15:11:01 EST       (caww01)\n#\n# Data for the following 1 site(s) are contained in this file\n#    USGS 12178000 SKAGIT RIVER AT NEWHALEM, WA\n# -----------------------------------------

In [10]:
# Getting temp and flow data from USGS
year = '2015'
url = 'https://waterdata.usgs.gov/nwis/dv?cb_00010=on&cb_00060=on&cb_00065=on&format=rdb&site_no=12178000&legacy=&referred_module=sw&period=&begin_date=2015-01-01&end_date=2015-12-16'

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()
text_data = text[1951:] # Keep anything after the text in position 1951 (where data starts)
df = pd.read_csv(io.StringIO(text_data), sep = '\t', header = None, names = ["USGS", "random0", "Date", "flow", "random1",
                                                                       "temp_max", "random2", 
                                                                       "temp_min", "random3", "temp", "random4"])

# # Double check that the column names are lined up accordingly
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df)


# Subset the data
df_temp_flow_subset = df[['Date','flow','temp']]

# Format the date in the subset
df_temp_flow_subset['Date'] = pd.to_datetime(df_temp_flow_subset['Date'])

# Display the temp_flow subset
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_temp_flow_subset)

/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/4199851369.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_flow_subset['Date'] = pd.to_datetime(df_temp_flow_subset['Date'])


,Date,flow,temp
0,2015-01-02,NaN,5.3
1,2015-01-03,NaN,5.3
2,2015-01-04,NaN,5.3
3,2015-01-05,NaN,5.1
4,2015-01-06,NaN,5.1
5,2015-01-07,NaN,5.2
6,2015-01-08,NaN,5.2
7,2015-01-09,NaN,5.1
8,2015-01-10,NaN,5.1
9,2015-01-11,NaN,5.2


In [11]:
# #------------------------------------------------------------------------------------------------------------------
# # Skip this entire step - use the saved csv (this cell was only added to show an example of the code)
# #------------------------------------------------------------------------------------------------------------------
# def photo_period_new(year):
#     hours_list = [] 
#     doy = range(1,366)
#     for days in doy:
    
#         i = datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)
#         date = i.strftime("%Y-%m-%d")
#         url = "https://api.sunrise-sunset.org/json?lat=48.422006&lng=-122.340316&date=" + date
#         response = requests.get(url)
#         result = response.json()
#         t = result['results']['day_length']
#         (h, m, s) = t.split(':')
#         hours = int(h) + int(m) * (1/60) #ignoring seconds
#         hours_list.append(hours)
#     df = pd.DataFrame({'doy' : doy, "photoperiod" : hours_list})
#     return(df)
# df_photo = photo_period_new(2015)

# display(df_photo)

# # Save into a csv to read later in order to save time
# df_photo.to_csv(data_string + "/photoperiod_skagit.csv")

In [12]:
# Skip the entire step in the cell above by reading the saved csv (since the code takes awhile to run)
df_photo = pd.read_csv(data_string + "/photoperiod_skagit.csv")

In [13]:
# Calculating the cumulative sum of the 'temp' 
df_temp_flow_subset['atu_solstice'] = df_temp_flow_subset['temp'].cumsum() # Accumulates sum of 'temp' values
year = df_temp_flow_subset['Date'].dt.year[0]
atu_df_april = df_temp_flow_subset.loc[(df_temp_flow_subset['Date'] >= str(year)+'-4-01')] # New df from subset for April
atu_df_april['atu_april'] = atu_df_april['temp'].cumsum() # Accumulates the sum of 'temp' column values beginning in April


# Merging subset & April atu df
df_atu = df_temp_flow_subset.merge(atu_df_april[['Date','atu_april']], how = 'left', on='Date') # The merging is performed on a left-join basis, preserving all rows from the subset
df_atu['flow_diff'] = df_atu['flow'].diff() # Calculating difference in flow
df_atu['temp_diff'] = df_atu['temp'].diff() # Calculating difference in temp
df_atu['doy'] = df_atu['Date'].dt.dayofyear # Calculating the day of the year


# Adding photoperiod to atu df
df_atu_photo = df_atu.merge(df_photo, how = 'left', on='doy') # Again, merging by left-join basis = preserving all rows from the df


# Creating a lunar df
df_lunar = pd.DataFrame(pd.date_range(start='2014-12-21', end='2015-12-21'), columns = ['Date'])
df_lunar.head()

# Creating a moon list by computing the moon phase (illuminated percentage of the moon)
moon_list = []
for i in df_lunar['Date']:
    a=ephem.Moon(i)
    mn=a.moon_phase # Illuminated percentage of the moon.
    moon_list.append(mn)

# Adding lunar phase to df_lunar based on the 'moon_list' from the code above, associating each phase with its corresponding date
df_lunar['lunar_phase'] = moon_list
df_lunar.head()


# Merging 'df_atu_photo' & 'df_lunar' into a new df
df_atu_photo_lunar = df_atu_photo.merge(df_lunar, how = 'left', on='Date') # Left-join basis = preserving all rows from the df
df_atu_photo_lunar['photo_diff'] = df_atu_photo_lunar['photoperiod'].diff() # Calculating difference in photoperiod


# Display all rows & columns of the df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #display(atu_df_april)
    #display(df_atu)
    #display(df_atu_photo)
    #display(df_lunar)
    display(df_atu_photo_lunar)

/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/2719585782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_flow_subset['atu_solstice'] = df_temp_flow_subset['temp'].cumsum() # Accumulates sum of 'temp' values
/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/2719585782.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atu_df_april['atu_april'] = atu_df_april['temp'].cumsum() # Accumulates the sum of 'temp' column values beginning in April


,Date,flow,temp,atu_solstice,atu_april,flow_diff,temp_diff,doy,Unnamed: 0,photoperiod,lunar_phase,photo_diff
0,2015-01-02,NaN,5.3,5.3,NaN,NaN,NaN,2,1,8.483333,0.896287,NaN
1,2015-01-03,NaN,5.3,10.6,NaN,NaN,0.0,3,2,8.500000,0.949709,0.016667
2,2015-01-04,NaN,5.3,15.9,NaN,NaN,0.0,4,3,8.516667,0.983669,0.016667
3,2015-01-05,NaN,5.1,21.0,NaN,NaN,-0.2,5,4,8.533333,0.997700,0.016667
4,2015-01-06,NaN,5.1,26.1,NaN,NaN,0.0,6,5,8.566667,0.992107,0.033333
5,2015-01-07,NaN,5.2,31.3,NaN,NaN,0.1,7,6,8.583333,0.967920,0.016667
6,2015-01-08,NaN,5.2,36.5,NaN,NaN,0.0,8,7,8.616667,0.926767,0.033333
7,2015-01-09,NaN,5.1,41.6,NaN,NaN,-0.1,9,8,8.633333,0.870688,0.016667
8,2015-01-10,NaN,5.1,46.7,NaN,NaN,0.0,10,9,8.666667,0.801910,0.033333
9,2015-01-11,NaN,5.2,51.9,NaN,NaN,0.1,11,10,8.700000,0.722664,0.033333


In [14]:
# Merging df's to include covariates for day & night
df_day_covariates = df_merged_day.merge(df_atu_photo_lunar, on = ['Date','doy'], how = 'left', validate = 'many_to_one') # 'validate' parameter checks whether the merge is a many-to-one relationship
df_night_covariates = df_merged_night.merge(df_atu_photo_lunar, on = ['Date', 'doy'], how = 'left', validate = 'many_to_one')

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #display(df_2015_scoop)
    #display(df_year_subset)
    #display(df_merged_day)
    #display(df_merged_night)
    #display(df_day_covariates)
    display(df_night_covariates)

,date_range,Down_time,Up_date,Up_time,midpoint,In,Up,Down,chinook0_wild_num,chinook0_hatchery_num,chinook1_wild_num,chinook1_hatchery_num,chum0_wild_num,coho0_wild_num,coho1_wild_num,coho1_hatchery_num,pink0_wild_num,sockeye1_wild_num,sockeye0_wild_num,steelheadsmolt_wild_num,daytime_category,midpoint_doy,doy,year,Date,flow,temp,atu_solstice,atu_april,flow_diff,temp_diff,Unnamed: 0,photoperiod,lunar_phase,photo_diff
0,2015-01-28,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,night,NaN,28,2015,2015-01-28,NaN,5.1,139.0,NaN,NaN,0.1,27,9.366667,0.587265,0.050000
1,2015-01-29,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,night,NaN,29,2015,2015-01-29,NaN,4.9,143.9,NaN,NaN,-0.2,28,9.416667,0.688251,0.050000
2,2015-01-30,1899-12-30 17:30:00,2015-01-30,1899-12-30 07:00:00,2015-01-30 00:15:00,13.500000,2015-01-30 07:00:00,2015-01-29 17:30:00,101.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,30.0,30,2015,2015-01-30,NaN,4.8,148.7,NaN,NaN,-0.1,29,9.466667,0.778613,0.050000
3,2015-01-31,1899-12-30 17:30:00,2015-01-31,1899-12-30 07:30:00,2015-01-31 00:30:00,14.000000,2015-01-31 07:30:00,2015-01-30 17:30:00,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,31.0,31,2015,2015-01-31,NaN,4.9,153.6,NaN,NaN,0.1,30,9.516667,0.855681,0.050000
4,2015-02-01,1899-12-30 17:30:00,2015-02-01,1899-12-30 08:00:00,2015-02-01 00:45:00,14.500000,2015-02-01 08:00:00,2015-01-31 17:30:00,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,32.0,32,2015,2015-02-01,NaN,4.9,158.5,NaN,NaN,0.0,31,9.550000,0.917457,0.033333
5,2015-02-02,1899-12-30 20:30:00,2015-02-02,1899-12-30 08:00:00,2015-02-02 02:15:00,11.500000,2015-02-02 08:00:00,2015-02-01 20:30:00,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,33.0,33,2015,2015-02-02,NaN,4.9,163.4,NaN,NaN,0.0,32,9.600000,0.962461,0.050000
6,2015-02-03,1899-12-30 17:30:00,2015-02-03,1899-12-30 08:00:00,2015-02-03 00:45:00,14.500000,2015-02-03 08:00:00,2015-02-02 17:30:00,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,34.0,34,2015,2015-02-03,NaN,5.0,168.4,NaN,NaN,0.1,33,9.650000,0.989692,0.050000
7,2015-02-04,1899-12-30 17:30:00,2015-02-04,1899-12-30 07:30:00,2015-02-04 00:30:00,14.000000,2015-02-04 07:30:00,2015-02-03 17:30:00,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,35.0,35,2015,2015-02-04,NaN,5.0,173.4,NaN,NaN,0.0,34,9.716667,0.998675,0.066667
8,2015-02-05,1899-12-30 17:45:00,2015-02-05,1899-12-30 08:00:00,2015-02-05 00:52:30,14.250000,2015-02-05 08:00:00,2015-02-04 17:45:00,51.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,night,36.0,36,2015,2015-02-05,NaN,5.1,178.5,NaN,NaN,0.1,35,9.766667,0.989519,0.050000
9,2015-02-06,1899-12-30 17:30:00,2015-02-06,1899-12-30 03:00:00,2015-02-05 22:15:00,9.500000,2015-02-06 03:00:00,2015-02-05 17:30:00,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,36.0,37,2015,2015-02-06,NaN,5.1,183.6,NaN,NaN,0.0,36,9.816667,0.962907,0.050000


In [16]:
# Save the scoop w/ covariates as a csv
df_day_covariates.to_csv(output_string+'/2015_skagit_scoop_covariates_day.csv')
df_night_covariates.to_csv(output_string+'/2015_skagit_scoop_covariates_night.csv')

---------------------------------------------------------------------------------------------------------------------
                                                    For screw file!
---------------------------------------------------------------------------------------------------------------------

In [17]:
# Read the dataset

year = 2015
csv_name = data_string + str(year) + "_skagit_mainstem_screw.csv"
df_2015_screw = pd.read_csv(csv_name, header = 0)

df_2015_screw.head()

,Unnamed: 0,StartDate,EndDate,StartTime,EndTime,chinook0_wild_num,chinook0_hatchery_num,chinook1_wild_num,chum0_wild_num,coho0_wild_num,coho1_wild_num,coho1_hatchery_num,steelheadsmolt_wild_num,sockeye0_wild_num,sockeye1_wild_num,pink0_wild_num,start_datetime,end_datetime,In,midpoint
0,1,2015-01-28,2015-01-28,1899-12-30 12:30:00,1899-12-30 17:30:00,9,0,0,0,0,0,0,0,0,0,0,2015-01-28 12:30:00,2015-01-28 17:30:00,5.0,2015-01-28 15:00:00
1,2,2015-01-29,2015-01-30,1899-12-30 17:30:00,1899-12-30 07:00:00,79,0,0,0,0,0,0,0,0,0,0,2015-01-29 17:30:00,2015-01-30 07:00:00,13.5,2015-01-30 00:15:00
2,3,2015-01-30,2015-01-31,1899-12-30 17:30:00,1899-12-30 07:30:00,98,0,0,0,0,0,0,0,0,0,0,2015-01-30 17:30:00,2015-01-31 07:30:00,14.0,2015-01-31 00:30:00
3,4,2015-01-31,2015-02-01,1899-12-30 17:30:00,1899-12-30 08:00:00,41,0,0,0,0,0,0,0,0,0,0,2015-01-31 17:30:00,2015-02-01 08:00:00,14.5,2015-02-01 00:45:00
4,5,2015-02-01,2015-02-02,1899-12-30 20:30:00,1899-12-30 08:00:00,22,0,0,0,0,0,0,0,0,0,0,2015-02-01 20:30:00,2015-02-02 08:00:00,11.5,2015-02-02 02:15:00


In [18]:
# Rename columns
df_year = df_2015_screw.rename(columns={'EndDate': "Up_date",
                                        'StartTime': "Down_time",
                                        'EndTime': "Up_time",
                                        'start_datetime': "Down",
                                        'end_datetime': "Up"})

df_year.columns   # List the columns

Index(['Unnamed: 0', 'StartDate', 'Up_date', 'Down_time', 'Up_time',
       'chinook0_wild_num', 'chinook0_hatchery_num', 'chinook1_wild_num',
       'chum0_wild_num', 'coho0_wild_num', 'coho1_wild_num',
       'coho1_hatchery_num', 'steelheadsmolt_wild_num', 'sockeye0_wild_num',
       'sockeye1_wild_num', 'pink0_wild_num', 'Down', 'Up', 'In', 'midpoint'],
      dtype='object')

In [19]:
# Subset the data
df_year_subset = df_year[['Down_time', 'Up_date', 'Up_time', 'midpoint', 'In', 'Up', 'Down',
                          'chinook0_wild_num', 'chinook0_hatchery_num',
                          'chinook1_wild_num', 
                          #'chinook1_hatchery_num',
                          'chum0_wild_num',
                          'coho0_wild_num',
                          'coho1_wild_num', 'coho1_hatchery_num',
                          'pink0_wild_num',
                          'sockeye1_wild_num', 'sockeye0_wild_num',
                          'steelheadsmolt_wild_num']]

In [20]:
# Add " 00:00:00" since it was removed
df_year_subset.loc[52, 'midpoint'] += " 00:00:00"

# Format 'midpoint' to a date-time stamp
df_year_subset['midpoint'] = pd.to_datetime(df_year_subset['midpoint'])

# Checking whether the midpoint of the time when trap was in during day or night
def categorize_daytime(dt):
    if dt.hour >= 6 and dt.hour < 18:
        return 'Day'
    else:
        return 'Night'
df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)

/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/3755800182.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year_subset['midpoint'] = pd.to_datetime(df_year_subset['midpoint'])
/var/folders/bx/0y_fwx6x0k33zy5f5rw2y4_r0000gn/T/ipykernel_65989/3755800182.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)


In [21]:
df_day = df_year_subset[df_year_subset['daytime_category']=="Day"]
df_night = df_year_subset[df_year_subset['daytime_category']=="Night"]

# Need to have one row for every date
df_dates_day = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2015-01-28 12:00:00', 
                                       end='2015-07-14 12:00:00')})
df_dates_night = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2015-01-28 00:00:00', 
                                       end='2015-07-14 00:00:00')})

df_merged_day = pd.merge_asof(df_dates_day, 
              df_day, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

df_merged_night = pd.merge_asof(df_dates_night, 
              df_night, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

In [22]:
# New columns
df_merged_day['midpoint_doy'] = df_merged_day['midpoint'].dt.dayofyear
df_merged_day['doy'] = df_merged_day['date_range'].dt.dayofyear
df_merged_day['year'] = df_merged_day['date_range'].dt.year
df_merged_day['daytime_category'] = 'day'
df_merged_day['Date'] = pd.to_datetime(df_merged_day['date_range'].dt.date)

df_merged_night['midpoint_doy'] = df_merged_night['midpoint'].dt.dayofyear
df_merged_night['doy'] = df_merged_night['date_range'].dt.dayofyear
df_merged_night['year'] = df_merged_night['date_range'].dt.year
df_merged_night['daytime_category'] = 'night'
df_merged_night['Date'] = pd.to_datetime(df_merged_night['date_range'].dt.date)

In [23]:
# Merging df's to include covariates for day & night
df_day_covariates = df_merged_day.merge(df_atu_photo_lunar, on = ['Date','doy'], how = 'left', validate = 'many_to_one') # 'validate' parameter checks whether the merge is a many-to-one relationship
df_night_covariates = df_merged_night.merge(df_atu_photo_lunar, on = ['Date', 'doy'], how = 'left', validate = 'many_to_one')

In [24]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #display(df_2015_screw)
    #display(df_year_subset)
    #display(df_merged_day)
    #display(df_merged_night)
    #display(df_day_covariates)
    display(df_night_covariates)

,date_range,Down_time,Up_date,Up_time,midpoint,In,Up,Down,chinook0_wild_num,chinook0_hatchery_num,chinook1_wild_num,chum0_wild_num,coho0_wild_num,coho1_wild_num,coho1_hatchery_num,pink0_wild_num,sockeye1_wild_num,sockeye0_wild_num,steelheadsmolt_wild_num,daytime_category,midpoint_doy,doy,year,Date,flow,temp,atu_solstice,atu_april,flow_diff,temp_diff,Unnamed: 0,photoperiod,lunar_phase,photo_diff
0,2015-01-28,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,night,NaN,28,2015,2015-01-28,NaN,5.1,139.0,NaN,NaN,0.1,27,9.366667,0.587265,0.050000
1,2015-01-29,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,night,NaN,29,2015,2015-01-29,NaN,4.9,143.9,NaN,NaN,-0.2,28,9.416667,0.688251,0.050000
2,2015-01-30,1899-12-30 17:30:00,2015-01-30,1899-12-30 07:00:00,2015-01-30 00:15:00,13.500000,2015-01-30 07:00:00,2015-01-29 17:30:00,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,30.0,30,2015,2015-01-30,NaN,4.8,148.7,NaN,NaN,-0.1,29,9.466667,0.778613,0.050000
3,2015-01-31,1899-12-30 17:30:00,2015-01-31,1899-12-30 07:30:00,2015-01-31 00:30:00,14.000000,2015-01-31 07:30:00,2015-01-30 17:30:00,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,31.0,31,2015,2015-01-31,NaN,4.9,153.6,NaN,NaN,0.1,30,9.516667,0.855681,0.050000
4,2015-02-01,1899-12-30 17:30:00,2015-02-01,1899-12-30 08:00:00,2015-02-01 00:45:00,14.500000,2015-02-01 08:00:00,2015-01-31 17:30:00,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,32.0,32,2015,2015-02-01,NaN,4.9,158.5,NaN,NaN,0.0,31,9.550000,0.917457,0.033333
5,2015-02-02,1899-12-30 20:30:00,2015-02-02,1899-12-30 08:00:00,2015-02-02 02:15:00,11.500000,2015-02-02 08:00:00,2015-02-01 20:30:00,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,33.0,33,2015,2015-02-02,NaN,4.9,163.4,NaN,NaN,0.0,32,9.600000,0.962461,0.050000
6,2015-02-03,1899-12-30 17:30:00,2015-02-03,1899-12-30 08:00:00,2015-02-03 00:45:00,14.500000,2015-02-03 08:00:00,2015-02-02 17:30:00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,34.0,34,2015,2015-02-03,NaN,5.0,168.4,NaN,NaN,0.1,33,9.650000,0.989692,0.050000
7,2015-02-04,1899-12-30 17:30:00,2015-02-04,1899-12-30 07:30:00,2015-02-04 00:30:00,14.000000,2015-02-04 07:30:00,2015-02-03 17:30:00,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,35.0,35,2015,2015-02-04,NaN,5.0,173.4,NaN,NaN,0.0,34,9.716667,0.998675,0.066667
8,2015-02-05,1899-12-30 17:45:00,2015-02-05,1899-12-30 08:00:00,2015-02-05 00:52:30,14.250000,2015-02-05 08:00:00,2015-02-04 17:45:00,34.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,36.0,36,2015,2015-02-05,NaN,5.1,178.5,NaN,NaN,0.1,35,9.766667,0.989519,0.050000
9,2015-02-06,1899-12-30 17:30:00,2015-02-06,1899-12-30 07:15:00,2015-02-06 00:22:30,13.750000,2015-02-06 07:15:00,2015-02-05 17:30:00,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,night,37.0,37,2015,2015-02-06,NaN,5.1,183.6,NaN,NaN,0.0,36,9.816667,0.962907,0.050000


In [25]:
# Save the screw w/ covariates as a csv
df_day_covariates.to_csv(output_string+'/2015_skagit_screw_covariates_day.csv')
df_night_covariates.to_csv(output_string+'/2015_skagit_screw_covariates_night.csv')